# Collective Endowment Effects ABM

In [4]:
import random
import warnings

import polars as pl
import altair as alt
from mesa import Agent
from mesa import Model
from mesa.time import RandomActivation
from mesa.space import NetworkGrid
import networkx as nx

warnings.filterwarnings("ignore", category=DeprecationWarning)
random.seed("cee")

Could not import SolaraViz. If you need it, install with 'pip install --pre mesa[viz]'


In [384]:
class CommunityAgent(Agent):
    def __init__(self, unique_id, model, asset_probability=0.5):
        # Initialize the agent
        self.unique_id = unique_id
        self.model = model
        self.pos = None
        self.money = model.random.randint(10, 100)
        self.assets = (
            model.random.randint(1, 10) if model.random.random() < asset_probability else 0
        )
        self.social_ties = {}  # Relationships with other agents
        self.interaction_history = {}  # Track recent interactions
        self.trade_state = None
        self.trade_quantity = 0
        self.price_history = []  # Track the last 10 prices
        self.highest_wtp = 0  # Track the highest WTP
        self.lowest_wta = 0  # Track the lowest WTA
        self.wta_wtp_gap = 0  # Track the gap between WTA and WTP
        self.social_influence = 0
        self.willingness_to_pay = 0
        self.willingness_to_accept = 0
        
    def interact(self):
        """Interact with a neighbor and adjust social ties and interaction history."""
        neighbors = list(self.model.grid.get_neighbors(self.unique_id, include_center=False))

        if not neighbors:
            return  # No neighbors to interact with

        # Compute weights based on social ties
        weights = [
            max(0, 1 + self.social_ties.get(neighbor_id, 0)) for neighbor_id in neighbors
        ]
        total_weight = sum(weights)
        weights = [w / total_weight for w in weights] if total_weight > 0 else [1 / len(neighbors)] * len(neighbors)

        # Select a neighbor based on weights
        partner_id = self.model.random.choices(neighbors, weights=weights, k=1)[0]

        # Determine interaction outcome
        interaction_outcome = self.model.random.choice(['positive', 'negative'])

        # Update social tie strength
        tie_strength = self.social_ties.get(partner_id, 0)
        self.social_ties[partner_id] = tie_strength + 1 if interaction_outcome == 'positive' else tie_strength - 1

        # Update interaction history
        self.interaction_history[partner_id] = (
            1 if interaction_outcome == 'positive' else -1
        )  # Store +1 for positive, -1 for negative

    def compute_social_influence(self):
        """Compute influence from social ties, incorporating recent interactions."""
        social_influence = 0

        for neighbor_id, tie_strength in self.social_ties.items():
            neighbor = next((a for a in self.model.schedule.agents if a.unique_id == neighbor_id), None)
            if not neighbor:
                continue

            # Diminishing effect based on neighbor's asset ownership
            asset_influence = 0
            if neighbor.assets > 0:
                asset_influence = tie_strength * (1 + 1 / (1 + neighbor.assets))

            # Adjust for interaction history
            recent_interaction = self.interaction_history.get(neighbor_id, 0)  # Default to neutral
            if recent_interaction < 0:  # Negative interaction
                asset_influence *= (1 + recent_interaction)  # Reduce influence
            elif recent_interaction > 0:  # Positive interaction
                asset_influence *= (1 + recent_interaction)  # Enhance influence

            # Add or subtract influence based on ownership and interaction
            if neighbor.assets > 0:
                social_influence += max(0, asset_influence)  # Positive influence
            else:
                social_influence -= max(0, tie_strength * 0.1)  # Negative influence for non-owners

        return social_influence

    def update_price_history(self, current_price):
        """Update the agent's price history with the latest market price."""
        self.price_history.append(current_price)
        if len(self.price_history) > 10:
            self.price_history.pop(0)  # Keep only the last 10 prices

    def compute_wtp(self, price):
        """Compute willingness to pay (WTP) with a threshold adjustment."""
        scaling_factor = 0.1
        base_wtp = price * (1 + scaling_factor * self.social_influence - 0.001)

        # Check if price dropped by more than 5% over the last 10 steps
        if len(self.price_history) >= 10:
            max_price = max(self.price_history[:-1])  # Exclude the current price
            if (max_price - price) / max_price > 0.03:  # Price dropped >3%
                if self.assets > 0:
                    base_wtp *= 1 + 1 / (pow(self.assets, 2))  # Increase WTP with asset ownership
                    return min(base_wtp, self.highest_wtp)
                if self.assets == 0:
                    # Reduce WTP unless price exceeds previously highest WTP
                    return min(base_wtp, self.highest_wtp)

        # Update highest WTP if base WTP is higher
        self.highest_wtp = max(self.highest_wtp, base_wtp)
        return base_wtp

    def compute_wta(self, price):
        """Compute willingness to accept (WTA) as a function of current price and asset count."""
        if self.assets == 0:
            return 0  # No willingness to sell if no assets

        base_wta = price * (1 + 1 / (self.assets)  + self.social_influence + 0.001)

        if len(self.price_history) >= 10:
            min_price_wta = max(self.price_history[:-1])
            if min_price_wta > 0 and (price - min_price_wta) / min_price_wta > 0.05:
                if self.assets > 0:
                    # Increase WTA unless price is above the lowest price wta (previous max)
                    return max(base_wta, self.lowest_wta)

        self.lowest_wta = max(self.lowest_wta, base_wta)   
        return base_wta

    def is_neighbor_owner(self, neighbor_id):
        """Check if a neighbor owns assets."""
        neighbor = next((a for a in self.model.schedule.agents if a.unique_id == neighbor_id), None)
        return neighbor and neighbor.assets > 0

    def calculate_value_preference(self):
        """Calculate the agent's value preference based on WTP and WTA."""
        price = self.model.market_maker['price']
        self.social_influence = self.compute_social_influence()
        self.willingness_to_pay = self.compute_wtp(price)
        self.willingness_to_accept = self.compute_wta(price)
        self.wta_wtp_gap = self.willingness_to_accept - self.willingness_to_pay

    def decide_trade(self):
        """Decide whether to buy or sell, and the quantity."""
        # Update WTP and WTA dynamically
        self.calculate_value_preference()
        price = self.model.market_maker['price']

        if self.willingness_to_pay >= price and self.money > price:
            preferred_action = 'buy'
        elif self.assets > 0 and self.willingness_to_accept <= price:
            preferred_action = 'sell'
        else:
            # No trade if WTP/WTA do not justify it
            self.trade_state = None
            self.trade_quantity = 0
            return
        
        if self.random.random() > 0.5:

            if preferred_action == 'buy':
                # WTP is higher than market price, so buy
                self.trade_state = 'buy'
                max_affordable = int(self.money / price)
                self.trade_quantity = min(max_affordable, 10)  # Max of 10 assets per trade
            elif preferred_action == 'sell':
                self.trade_state = 'sell'
                self.trade_quantity = min(self.assets, 10)  # Max of 10 assets per trade

        else:
            # No trade if WTP/WTA do not justify it
            self.trade_state = None
            self.trade_quantity = 0

    def trade(self):
        """Perform trade with the market maker."""
        price = self.model.market_maker['price']
        if self.trade_state == 'buy' and self.trade_quantity > 0:
            total_cost = self.trade_quantity * price
            if self.money >= total_cost and self.model.market_maker['inventory'] >= self.trade_quantity:
                self.money -= total_cost
                self.assets += self.trade_quantity
                self.model.market_maker['inventory'] -= self.trade_quantity

        elif self.trade_state == 'sell' and self.trade_quantity > 0:
            total_revenue = self.trade_quantity * price
            if self.assets >= self.trade_quantity:
                self.money += total_revenue
                self.assets -= self.trade_quantity
                self.model.market_maker['inventory'] += self.trade_quantity

        # Update value preference after trading
        self.value_preference = self.calculate_value_preference()

    def step(self):
        """Run one step of the agent's behavior."""
        current_price = self.model.market_maker['price']
        self.update_price_history(current_price)
        self.calculate_value_preference()

        self.interact()
        self.decide_trade()
        self.trade()

In [385]:
class CommunityModel(Model):
    def __init__(self, num_agents):
        super().__init__()  # Initialize the parent Model class
        self.random = random.Random()  # Initialize the model's random generator
        self.num_agents = num_agents
        self.schedule = RandomActivation(self)

        # Define networks
        self.interaction_network = nx.newman_watts_strogatz_graph(n=num_agents, k=3, p=0.3)
        self.exchange_network = nx.star_graph(n=num_agents)  # Star network with market maker at the center
        self.grid = NetworkGrid(self.interaction_network)

        # Market maker
        self.market_maker = {'price': 10, 'inventory': 1000}

        # Historical data tracker
        self.history = []

        # Add agents
        for i in range(num_agents):
            agent = CommunityAgent(i, self, asset_probability=0.1)
            self.schedule.add(agent)
            if not self.grid.is_cell_empty(i):
                raise Exception("Grid cell is not empty")
            self.grid.place_agent(agent, i)

    def update_price(self):
        """Adjust price based on supply and demand."""
        total_buy = sum(agent.trade_quantity for agent in self.schedule.agents if agent.trade_state == 'buy')
        total_sell = sum(agent.trade_quantity for agent in self.schedule.agents if agent.trade_state == 'sell')

        if random.random() > 0.8:
            # shock the market
            self.market_maker['price'] += 2 * random.random() * random.choice([-1, 1])
        else:
            # Adjust price based on the net demand
            self.market_maker['price'] += 0.05 * (total_buy - total_sell)

        self.market_maker['price'] = max(self.market_maker['price'], 1)  # Ensure price stays positive

    def step(self):
        """Advance the model by one step."""
        # Activate agents
        self.schedule.step()
        # Update the market price
        self.update_price()

        # Initialize variables for tracking metrics
        total_assets = 0
        wtp_owners = []
        wta_owners = []
        wtp_non_owners = []
        wta_non_owners = []

        # Iterate through agents to calculate metrics
        for agent in self.schedule.agents:
            total_assets += agent.assets  # Sum total assets

            # Track WTP and WTA based on ownership
            if agent.assets > 0:
                wtp_owners.append(agent.willingness_to_pay)
                wta_owners.append(agent.willingness_to_accept)
            else:
                wtp_non_owners.append(agent.willingness_to_pay)
                wta_non_owners.append(agent.willingness_to_accept)

        # Compute averages
        avg_wtp_owners = sum(wtp_owners) / len(wtp_owners) if wtp_owners else 0
        avg_wta_owners = sum(wta_owners) / len(wta_owners) if wta_owners else 0
        avg_wtp_non_owners = sum(wtp_non_owners) / len(wtp_non_owners) if wtp_non_owners else 0
        avg_wta_non_owners = sum(wta_non_owners) / len(wta_non_owners) if wta_non_owners else 0
        avg_assets = total_assets / len(self.schedule.agents)  # Average assets per agent

        # Record history
        self.history.append({
            "step": len(self.history),
            "price": self.market_maker['price'],
            "average_assets": avg_assets,
            "avg_wtp_owners": avg_wtp_owners,
            "avg_wta_owners": avg_wta_owners,
            "avg_wtp_non_owners": avg_wtp_non_owners,
            "avg_wta_non_owners": avg_wta_non_owners,
        })


    def get_visualization_data(self):
        """Prepare data for Altair visualization."""
        # Nodes data
        nodes = []
        for agent in self.schedule.agents:
            nodes.append({
                "id": agent.unique_id,
                "assets": agent.assets,
                "wtp": agent.willingness_to_pay,
                "wta": agent.willingness_to_accept,
                "color": "orange" if agent.assets > 0 else "gray",  # Blue if owning assets
                "size": 20 + agent.assets * 30,  # Node size scales with assets
            })
        # Check nodes data
        print("Nodes Data:", nodes)
        nodes_df = pl.DataFrame(nodes)

        # Edges data
        edges = []
        for agent in self.schedule.agents:
            for neighbor, tie_strength in agent.social_ties.items():
                edges.append({
                    "source": agent.unique_id,
                    "target": neighbor.unique_id,
                    "tie_strength": tie_strength,
                    "color": "blue" if tie_strength > 0 else ("red" if tie_strength < 0 else "gray"),
                    "width": abs(tie_strength) * 0.5,  # Edge width scales with tie strength
                })
        # Check edges data
        print("Edges Data:", edges)
        edges_df = pl.DataFrame(edges)

        return nodes_df, edges_df

    def visualize_network(self):
        """Visualize the model using Altair with a force-directed layout."""
        nodes_df, edges_df = self.get_visualization_data()

        # Convert Polars DataFrames to Pandas for Altair compatibility
        nodes_df = nodes_df.to_pandas()
        edges_df = edges_df.to_pandas()

        # Create a NetworkX graph from the edges data
        G = nx.Graph()
        for _, edge in edges_df.iterrows():
            G.add_edge(edge["source"], edge["target"], weight=edge["tie_strength"])

        # Compute a force-directed layout for the graph
        pos = nx.spring_layout(G)

        # Add layout positions to the nodes DataFrame
        nodes_df["x"] = nodes_df["id"].apply(lambda node: pos[node][0])
        nodes_df["y"] = nodes_df["id"].apply(lambda node: pos[node][1])

        # Add layout positions to the edges DataFrame
        edges_df["x"] = edges_df["source"].apply(lambda node: pos[node][0])
        edges_df["y"] = edges_df["source"].apply(lambda node: pos[node][1])
        edges_df["x2"] = edges_df["target"].apply(lambda node: pos[node][0])
        edges_df["y2"] = edges_df["target"].apply(lambda node: pos[node][1])

        # Define edge layer
        edges_chart = alt.Chart(edges_df).mark_rule().encode(
            x="x:Q",
            x2="x2:Q",
            y="y:Q",
            y2="y2:Q",
            strokeWidth=alt.Size("width:Q", scale=alt.Scale(range=[1, 5]), legend=None),
            color=alt.Color("color:N", scale=None, legend=None),
        )

        # Define node layer
        nodes_chart = alt.Chart(nodes_df).mark_circle().encode(
            x="x:Q",
            y="y:Q",
            size=alt.Size("size:Q", scale=alt.Scale(range=[100, 1000]), legend=None),
            color=alt.Color("color:N", scale=None, legend=None),
            tooltip=["id", "assets", "wtp", "wta"],
        )

        # Combine node and edge layers
        chart = alt.layer(edges_chart, nodes_chart).configure_view(strokeWidth=0).configure_axis(grid=False)
        return chart

    def visualize_time_series(self):
        """Visualize average value preferences for owners and non-owners on the same chart, with a legend."""
        # Convert history to a Polars DataFrame
        history_df = pl.DataFrame(self.history)
        reshaped_df = history_df.select([
            pl.col("step").alias("Step"),
            pl.col("avg_wtp_owners"),
            pl.col("avg_wta_owners"),
            pl.col("avg_wtp_non_owners"),
            pl.col("avg_wta_non_owners"),
            pl.col("price"),
        ]).melt(
            id_vars=["Step"],
            value_vars=["avg_wtp_owners", "avg_wta_owners", "avg_wtp_non_owners", "avg_wtp_non_owners", "price"],
            variable_name="Group",
            value_name="Value"
        ).with_columns([
            pl.col("Step").cast(pl.Int32),
            pl.col("Value").cast(pl.Float64),
        ]).sort("Value")

        # Create the line chart for value preferences
        preference_chart = alt.Chart(reshaped_df).mark_line().encode(
            x=alt.X("Step:Q", title="Time Step"),
            y=alt.Y("Value:Q", title="Average Valuation"),
            color=alt.Color(
                "Group:N",
                scale=alt.Scale(domain=["avg_wtp_owners", "avg_wta_owners", "avg_wtp_non_owners", "avg_wtp_non_owners"], range=["blue", "green", "orange", "red"]),
                #scale=alt.Scale(domain=["Owners", "Non-Owners"], range=["blue", "green"]),
                title="Group"
            ),
        ).properties(width=600, height=300)

        # Create the secondary line chart for average assets
        assets_chart = alt.Chart(history_df).mark_point(filled=True, opacity=0.2).encode(
            x=alt.X("step:Q", title="Time Step"),
            y=alt.Y("average_assets:Q", title=None),
            color=alt.value("green"),
        )

        # Create the secondary line chart for average assets
        price_chart = alt.Chart(history_df).mark_line().encode(
            x=alt.X("step:Q", title="Time Step"),
            y=alt.Y("price:Q", title=None),
            color=alt.value("black"),
        )

        # Combine the charts
        combined_chart = alt.layer(preference_chart, price_chart).resolve_scale(
            y="independent"  # Independent scales for dual axes
        ).properties(title="Average Value Preferences (Owners vs Non-Owners) and Average Assets Over Time")

        return combined_chart


In [386]:
# Initialize the model with 16 agents
model = CommunityModel(num_agents=80)

# Run for 10 steps
for step in range(500):
    model.step()
    if step % 50 == 0:
        print(f"Step {step}: Price = {model.market_maker['price']:.2f}")


Step 0: Price = 10.00
Step 50: Price = 3.85
Step 100: Price = 4.43
Step 150: Price = 4.08
Step 200: Price = 3.03
Step 250: Price = 6.55
Step 300: Price = 9.25
Step 350: Price = 6.56
Step 400: Price = 3.67
Step 450: Price = 6.22


In [387]:
model.visualize_network()

Nodes Data: [{'id': 29, 'assets': 0, 'wtp': 4.865079861406341, 'wta': 0, 'color': 'gray', 'size': 20}, {'id': 57, 'assets': 0, 'wtp': 4.865079861406341, 'wta': 0, 'color': 'gray', 'size': 20}, {'id': 79, 'assets': 0, 'wtp': 4.865079861406341, 'wta': 0, 'color': 'gray', 'size': 20}, {'id': 61, 'assets': 0, 'wtp': 4.865079861406341, 'wta': 0, 'color': 'gray', 'size': 20}, {'id': 71, 'assets': 0, 'wtp': 4.865079861406341, 'wta': 0, 'color': 'gray', 'size': 20}, {'id': 25, 'assets': 14, 'wtp': 4.865079861406341, 'wta': 5.222673318972887, 'color': 'orange', 'size': 440}, {'id': 4, 'assets': 0, 'wtp': 4.865079861406341, 'wta': 0, 'color': 'gray', 'size': 20}, {'id': 9, 'assets': 0, 'wtp': 4.865079861406341, 'wta': 0, 'color': 'gray', 'size': 20}, {'id': 49, 'assets': 0, 'wtp': 4.865079861406341, 'wta': 0, 'color': 'gray', 'size': 20}, {'id': 23, 'assets': 0, 'wtp': 4.865079861406341, 'wta': 0, 'color': 'gray', 'size': 20}, {'id': 33, 'assets': 0, 'wtp': 4.865079861406341, 'wta': 0, 'color': 

alt.LayerChart(...)

In [388]:
model.visualize_time_series()

alt.LayerChart(...)

In [257]:
print(model.history[-1])

{'step': 499, 'price': 22.316100848177346, 'average_assets': 4.46875, 'avg_wtp_owners': 22.316100848177335, 'avg_wta_owners': 28.641506882828246, 'avg_wtp_non_owners': 22.316100848177346, 'avg_wta_non_owners': 0.0}


In [128]:
history_df = pl.DataFrame(model.history)

# Reshape the data for a single chart with a legend
history_df.select([
    pl.col("step").alias("Step"),
    pl.col("avg_wtp_owners"),
    pl.col("avg_wta_owners"),
    pl.col("avg_wtp_non_owners"),
    pl.col("avg_wta_non_owners"),
]).melt(
    id_vars=["Step"],
    value_vars=["avg_wtp_owners", "avg_wta_owners", "avg_wtp_non_owners", "avg_wtp_non_owners"],
    variable_name="Group",
    value_name="Value"
).with_columns([
    pl.col("Step").cast(pl.Int32),
    pl.col("Value").cast(pl.Float64),
]).sort("Value")

Step,Group,Value
i32,str,f64
0,"""avg_wtp_non_owners""",0.0
1,"""avg_wtp_non_owners""",0.0
2,"""avg_wtp_non_owners""",0.0
3,"""avg_wtp_non_owners""",0.0
4,"""avg_wtp_non_owners""",0.0
…,…,…
630,"""avg_wta_owners""",61.731079
626,"""avg_wta_owners""",62.244252
627,"""avg_wta_owners""",62.244252
